In [38]:
import os
import pandas as pd
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import os
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter
import plotly.express as px
import tensorboard as tb
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [39]:
# Show ALL rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Optional: prevent line wrapping
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


# Load tensorboard file after the training


In [40]:
# Functions usefull for the extraction of the information from the event file
def extract_scalars_from_event_file(log_dir):
    # Load TensorBoard event file
    event_acc = EventAccumulator(log_dir)
    event_acc.Reload()

    # List all scalar tags
    scalar_tags = event_acc.Tags().get("scalars", [])

    # Create a dictionary of DataFrames for each tag
    scalar_dataframes = {}

    for tag in scalar_tags:
        events = event_acc.Scalars(tag)
        df = pd.DataFrame({
            "wall_time": [e.wall_time for e in events],
            "step": [e.step for e in events],
            "value": [e.value for e in events],
        })
        scalar_dataframes[tag] = df

    return scalar_dataframes

def list_folders(directory):
    return [name for name in os.listdir(directory)
            if os.path.isdir(os.path.join(directory, name))]

variables = ['rollout/ep_len_mean', 'rollout/ep_rew_mean']

Plot figures

In [41]:
plt.figure()
# Create dataframes
df = pd.DataFrame(columns = ["run", "step", "len", "reward", "algorithm", "grid"])
df_file = pd.DataFrame(columns = ["run", "step", "len", "reward", "algorithm", "grid"])
algorithms = ["specs", "vanilla"]

#Grid to test
grid = 12

directory_path = f"./tb_logs/grid_{grid}/"
len_dir = len(list_folders(directory_path))
print("list_folders", list_folders(directory_path))

for file in list_folders(directory_path):
    dataframes = extract_scalars_from_event_file(directory_path + file)
    df_file["step"] = dataframes['rollout/ep_len_mean']["step"]
    df_file["len"] = dataframes['rollout/ep_len_mean']["value"]
    df_file["reward"] = dataframes['rollout/ep_rew_mean']["value"]


    if isinstance(file[5], int):
        df_file["run"] = int(file[5])
    else:
        df_file["run"] = int(file[7])
    df_file["algorithm"] = "specs" if file[:5] == "specs" else "vanilla"
    df_file["grid"] = grid
    
    df = pd.concat([df,df_file], ignore_index = True)

#print(df)

# === Aggregate Data ===
agg_df = (
    df.groupby(["algorithm", "step"])
    .agg(
        mean_reward=("reward", "mean"),
        std_reward=("reward", "std"),
        mean_len=("len", "mean"),
        std_len=("len", "std"),
    )
    .reset_index()
)

colors = {"vanilla": "royalblue", "specs": "darkorange"}

list_folders ['specs6_2025_10_22_1', 'vanilla5_2025_10_21_1', 'vanilla0_2025_10_17', 'vanilla6_2025_10_21_1', 'specs7_2025_10_22_1', 'specs9_2025_10_23_1', 'vanilla2_2025_10_17', 'vanilla8_2025_10_21_1', 'specs1_2025_10_17', 'vanilla7_2025_10_21_1', 'vanilla4_2025_10_17', 'vanilla1_2025_10_21_1', 'vanilla3_2025_10_17', 'specs5_2025_10_22_1', 'specs0_2025_10_17']


/tmp/ipykernel_502219/158658704.py:28: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



<Figure size 640x480 with 0 Axes>

In [42]:

# ============================================================
# 1️⃣  FIGURE 1 — Average Episode Reward
# ============================================================
fig_reward = go.Figure()

for algorithm in agg_df["algorithm"].unique():
    subset = agg_df[agg_df["algorithm"] == algorithm]

    # Mean line
    fig_reward.add_trace(
        go.Scatter(
            x=subset["step"],
            y=subset["mean_reward"],
            mode="lines",
            name=algorithm,
            line=dict(color=colors.get(algorithm, "gray")),
        )
    )

    # Std shaded region
    fig_reward.add_trace(
        go.Scatter(
            x=list(subset["step"]) + list(subset["step"][::-1]),
            y=list(subset["mean_reward"] + subset["std_reward"]) +
              list(subset["mean_reward"] - subset["std_reward"])[::-1],
            fill="toself",
            fillcolor=f"rgba(65,105,225,0.15)" if algorithm == "vanilla" else "rgba(255,140,0,0.15)",
            line=dict(color="rgba(255,255,255,0)"),
            showlegend=False,
            hoverinfo="skip",
        )
    )

fig_reward.update_layout(
    #title="Training Performance: Average Episode Reward",
    xaxis_title="Training Steps",
    yaxis_title="Average Reward",
    template="plotly_white",
    font=dict(size=13),
    legend_title_text="Algorithm",
    height=500,
    width=950,
)

In [43]:

# ============================================================
# 2️⃣  FIGURE 2 — Average Episode Length
# ============================================================
fig_len = go.Figure()

for algorithm in agg_df["algorithm"].unique():
    subset = agg_df[agg_df["algorithm"] == algorithm]

    # Mean line
    fig_len.add_trace(
        go.Scatter(
            x=subset["step"],
            y=subset["mean_len"],
            mode="lines",
            name=algorithm,
            line=dict(color=colors.get(algorithm, "gray")),
        )
    )

    # Std shaded region
    fig_len.add_trace(
        go.Scatter(
            x=list(subset["step"]) + list(subset["step"][::-1]),
            y=list(subset["mean_len"] + subset["std_len"]) +
              list(subset["mean_len"] - subset["std_len"])[::-1],
            fill="toself",
            fillcolor=f"rgba(0,0,255,0.1)" if algorithm == "vanilla" else "rgba(255,140,0,0.1)",
            line=dict(color="rgba(255,255,255,0)"),
            showlegend=False,
            hoverinfo="skip",
        )
    )
fig_len.update_layout(
    #title="Training Performance: Average Episode Length",
    xaxis_title="Training Steps",
    yaxis_title="Average Episode Length",
    template="plotly_white",
    font=dict(size=13),
    legend_title_text="Algorithm",
    height=500,
    width=950,
)

# ============================================================
# SHOW BOTH
# ============================================================
fig_reward.show()
fig_len.show()